In [187]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn
import matplotlib as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import models, layers, Input
from sklearn.preprocessing import StandardScaler


In [188]:
train_data = pd.read_csv("Train_data.csv")
train_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [189]:
# Create label encoders
label_encoders = {
    'protocol_type': LabelEncoder(),
    'service': LabelEncoder(),
    'flag': LabelEncoder(),
    'class': LabelEncoder()
}

# Fit and transform the columns
for column in ['protocol_type', 'service', 'flag', 'class']:
    train_data[column] = label_encoders[column].fit_transform(train_data[column])

# Review transformed data
train_data.head(20)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,19,9,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,1
1,0,2,41,9,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,1
2,0,1,46,5,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0
3,0,1,22,9,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,1
4,0,1,22,9,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1
5,0,1,46,1,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,0
6,0,1,46,5,0,0,0,0,0,0,...,9,0.04,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0
7,0,1,46,5,0,0,0,0,0,0,...,15,0.06,0.07,0.00,0.00,1.00,1.00,0.00,0.00,0
8,0,1,48,5,0,0,0,0,0,0,...,23,0.09,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0
9,0,1,46,5,0,0,0,0,0,0,...,13,0.05,0.06,0.00,0.00,1.00,1.00,0.00,0.00,0


In [190]:
top_n_features = train_data.columns.drop('class') 

# Separate features (X) and target variable (y)
X = train_data[top_n_features]
y = train_data['class']

In [191]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [192]:
print('X_train:',np.shape(X_train))
print('y_train:',np.shape(y_train))
print('X_test:',np.shape(X_test))
print('y_test:',np.shape(y_test))

X_train: (17634, 41)
y_train: (17634,)
X_test: (7558, 41)
y_test: (7558,)


In [193]:
# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [194]:
# len(top_n_features) is the number of features in your input data
input_shape = (len(top_n_features),)

print(input_shape)

model_tf = models.Sequential()

# Use Input(shape) as the first layer
model_tf.add(Input(shape=input_shape))
model_tf.add(layers.Dense(16, activation='relu'))

(41,)


In [195]:
model_tf.add(Dense(1, activation='sigmoid'))

In [196]:
adam = keras.optimizers.Adam(learning_rate=0.001)

In [197]:
model_tf.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])

In [198]:
model_tf.fit(X_train, y_train, epochs=100)

Epoch 1/100


552/552 ━━━━━━━━━━━━━━━━━━━━ 1s 454us/step - accuracy: 0.9036 - loss: 0.2866
Epoch 2/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 447us/step - accuracy: 0.9685 - loss: 0.1008
Epoch 3/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 449us/step - accuracy: 0.9716 - loss: 0.0833
Epoch 4/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step - accuracy: 0.9760 - loss: 0.0664
Epoch 5/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step - accuracy: 0.9765 - loss: 0.0556
Epoch 6/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step - accuracy: 0.9801 - loss: 0.0490
Epoch 7/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 454us/step - accuracy: 0.9837 - loss: 0.0441
Epoch 8/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step - accuracy: 0.9855 - loss: 0.0418
Epoch 9/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 448us/step - accuracy: 0.9868 - loss: 0.0421
Epoch 10/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 453us/step - accuracy: 0.9875 - loss: 0.0388
Epoch 11/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step - accuracy: 0.9882 - loss: 0.0368
Epoch 12/100
552/552 ━━━━━━━━━━

In [199]:
loss_and_metrics = model_tf.evaluate(X_test, y_test)
print(loss_and_metrics)
print('Loss = ',loss_and_metrics[0])
print(f'Test Accuracy: {loss_and_metrics[1] * 100:.2f}%')

237/237 ━━━━━━━━━━━━━━━━━━━━ 0s 408us/step - accuracy: 0.9934 - loss: 0.0306
[0.035499077290296555, 0.9933844804763794]
Loss =  0.035499077290296555
Test Accuracy: 99.34%


In [200]:
# Evaluate your model on the test set
y_pred = model_tf.predict(X_test)
y_pred_binary = np.round(y_pred)  # Convert probabilities to binary predictions

# Calculate and print the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_binary)
print("Confusion Matrix:")
print(conf_matrix)

237/237 ━━━━━━━━━━━━━━━━━━━━ 0s 494us/step
Confusion Matrix:
[[3496   20]
 [  30 4012]]


In [201]:
# Assuming your raw data does not include the target variable and follows the same order as your training data
raw_data = [0,"icmp","eco_i","SF",18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,16,1,0,1,1,0,0,0,0,]

# Convert the raw data to a pandas DataFrame
column_names = top_n_features  # Make sure this includes all feature names except 'class'
test_data = pd.DataFrame([raw_data], columns=column_names)

# Apply the same preprocessing steps (label encoding and scaling)
for column in ['protocol_type', 'service', 'flag']:
    try:
        test_data[column] = label_encoders[column].transform(test_data[column])
    except KeyError as e:
        print(f"KeyError: {e}. The value might not have been seen during training.")

# Scale the features
test_data_scaled = scaler.transform(test_data)

# Make a prediction
prediction = model_tf.predict(test_data_scaled)
predicted_class = (prediction > 0.5).astype("int32")

# Print the predicted class
if predicted_class[0][0] == 1:
    print("Predicted class: Normal")
else:
    print("Predicted class: Anomaly")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted class: Anomaly


In [202]:
# Assuming you have loaded the validation test set similar to the training data
validation_data = pd.read_csv("Test_data.csv")

# Create label encoders
label_encoders = {
    'protocol_type': LabelEncoder(),
    'service': LabelEncoder(),
    'flag': LabelEncoder(),
}

# Fit and transform the columns
for column in ['protocol_type', 'service', 'flag']:
    validation_data[column] = label_encoders[column].fit_transform(validation_data[column])

# Extract features from the validation test set
X_validation = validation_data[top_n_features]

# Make predictions on the validation test set
predictions = model_tf.predict(X_validation)

# Assuming you want to use clustering for evaluation
# You can choose a threshold to classify predictions into normal (0) or anomaly (1)
threshold = 0.5
predicted_labels = (predictions > threshold).astype("int32")

# Example of using silhouette score (assuming you have ground truth labels)
from sklearn.metrics import silhouette_score

# silhouette_score requires true labels, replace 'true_labels' with actual labels if available
# silhouette_score ranges from -1 to 1, higher values indicate better separation
silhouette_avg = silhouette_score(X_validation, predicted_labels)
print(f"Silhouette Score: {silhouette_avg}")


705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step


c:\Users\wasif\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Silhouette Score: -0.4989614375682268
